In [1]:
import os
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

from src.dataset import prep_dataloader
from src.model import NeuralNetwork
from src.training import train, test

In [2]:
# config
# path
tr_path = '../../data/processed/training-set.csv'
val_path = '../../data/processed/validation-set.csv'
tt_path = '../../data/processed/testing-set.csv'
log_path = '../../logs/attack_classification'
model_path = '../../models/attack_classification/model_weights.pth'

# hyperparameter
batch_size = 64

In [3]:
# prepare dataloader
tr_dl = prep_dataloader(
    tr_path,
    batch_size,
    shuffle=True
)

val_dl = prep_dataloader(
    val_path,
    batch_size,
    shuffle=True
)

tt_dl = prep_dataloader(
    tt_path,
    batch_size,
    shuffle=False
)

Finished reading the training set of Dataset (164910 samples found, each dim = 59)
Finished reading the validation set of Dataset (41228 samples found, each dim = 59)
Finished reading the testing set of Dataset (51535 samples found, each dim = 59)


In [7]:
# training and validation
# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# summary writer
last_log_path = os.path.join(log_path, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
writer = SummaryWriter(last_log_path)

# neural network
model = NeuralNetwork().to(device)

# load model weight
if os.path.isfile(model_path):
    print(f'Load model weights form {model_path}')
    model.load_state_dict(torch.load(model_path))

# loss func and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)1
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# training and validation
epochs = 1000
for t in range(epochs):
    ep = t + 1
    print(f"Epoch {ep}")
    tr_loss, tr_acc = train(tr_dl, model, loss_fn, optimizer, device)
    val_loss, val_acc = test(val_dl, model, loss_fn, device)
    
    # log loss and acc
    writer.add_scalar('Loss/train', tr_loss, ep)
    writer.add_scalar('Accuracy/train', tr_acc, ep)
    writer.add_scalar('Loss/val', val_loss, ep)
    writer.add_scalar('Accuracy/val', val_acc, ep)

# saving model
print(f'Saving model to {model_path}\n')
torch.save(model.state_dict(), model_path)
    
print("Done!")

Using cuda device
Load model weights form ../../models/attack_classification/model_weights.pth
Epoch 1


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 79.3%, Avg loss: 0.509964 

[Test] Accuracy: 79.4%, Avg loss: 0.524921 

Epoch 2


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 79.3%, Avg loss: 0.508702 

[Test] Accuracy: 78.6%, Avg loss: 0.526591 

Epoch 3


...

[Train] Accuracy: 84.2%, Avg loss: 0.375728 

[Test] Accuracy: 81.8%, Avg loss: 0.460024 

Epoch 999


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 84.1%, Avg loss: 0.376050 

[Test] Accuracy: 81.7%, Avg loss: 0.527136 

Epoch 1000


  0%|          | 0/2577 [00:00<?, ?it/s]

[Train] Accuracy: 84.1%, Avg loss: 0.376802 

[Test] Accuracy: 81.4%, Avg loss: 0.481488 

Saving model to ../../models/attack_classification/model_weights.pth

Done!


In [9]:
# testing
tt_loss, tt_acc = test(tt_dl, model, loss_fn, device)
writer.add_scalar('Loss/test', tt_loss, 1)
writer.add_scalar('Accuracy/test', tt_acc, 1)

[Test] Accuracy: 79.1%, Avg loss: 0.520311 

